In [31]:
# Build Recurrent Neural Network by Using the Numpy Library

In [32]:
import numpy as np

In [33]:
# RNN configuration
input_dim = 1
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
sequence_length = 5
num_epochs = 500

In [34]:
# Initial weights and biases

In [35]:
weight_input_hidden = np.random.randn(hidden_dim, input_dim) * 0.01
weight_hidden_hidden = np.random.randn(hidden_dim, hidden_dim) * 0.01
weight_hidden_output = np.random.randn(output_dim, hidden_dim) * 0.01

bias_hidden = np.zeros((hidden_dim, 1))
bias_output = np.zeros((output_dim, 1))

In [36]:
# Activation function and its derivative

In [37]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [38]:
def sigmoid_derivative(output):
    return output * (1 - output)

In [39]:
# Forward pass

def forward_step(input_t, hidden_state_prev):
    hidden_state = sigmoid(np.dot(weight_input_hidden, input_t) + np.dot(weight_hidden_hidden, hidden_state_prev) + bias_hidden)
    output = np.dot(weight_hidden_output, hidden_state) + bias_output
    return output, hidden_state

In [45]:
# Backpropagation Through Time (BPTT)


def bptt(inputs, targets, initial_hidden_state):
    global weight_input_hidden, weight_hidden_hidden, weight_hidden_output, bias_hidden, bias_output
    
    # Initialize gradients
    grad_weight_input_hidden = np.zeros_like(weight_input_hidden)
    grad_weight_hidden_hidden = np.zeros_like(weight_hidden_hidden)
    grad_weight_hidden_output = np.zeros_like(weight_hidden_output)
    grad_bias_hidden = np.zeros_like(bias_hidden)
    grad_bias_output = np.zeros_like(bias_output)
    grad_hidden_state_next = np.zeros_like(initial_hidden_state)
    
    # Store hidden states and outputs
    hidden_states, predicted_outputs = [np.copy(initial_hidden_state)], []
    
    # Forward pass through each time step
    hidden_state = initial_hidden_state
    for input_t in inputs:
        output, hidden_state = forward_step(input_t, hidden_state)
        predicted_outputs.append(output)
        hidden_states.append(hidden_state)
    
    # Backward pass through time
    for t in reversed(range(len(inputs))):
        output_error = predicted_outputs[t] - targets[t]
        grad_weight_hidden_output += np.dot(output_error, hidden_states[t].T)
        grad_bias_output += output_error
        hidden_state_error = np.dot(weight_hidden_output.T, output_error) + grad_hidden_state_next
        hidden_state_delta = sigmoid_derivative(hidden_states[t]) * hidden_state_error
        grad_bias_hidden += hidden_state_delta
        grad_weight_input_hidden += np.dot(hidden_state_delta, inputs[t].T)
        grad_weight_hidden_hidden += np.dot(hidden_state_delta, hidden_states[t-1].T)
        grad_hidden_state_next = np.dot(weight_hidden_hidden.T, hidden_state_delta)
    
    # Gradient clipping and parameter update
    for grad in [grad_weight_input_hidden, grad_weight_hidden_hidden, grad_weight_hidden_output, grad_bias_hidden, grad_bias_output]:
        np.clip(grad, -1, 1, out=grad)
        
    weight_input_hidden -= learning_rate * grad_weight_input_hidden
    weight_hidden_hidden -= learning_rate * grad_weight_hidden_hidden
    weight_hidden_output -= learning_rate * grad_weight_hidden_output
    bias_hidden -= learning_rate * grad_bias_hidden
    bias_output -= learning_rate * grad_bias_output
    
    return predicted_outputs, targets

In [46]:
# Accuracy Calculation
def calculate_accuracy(predicted_outputs, targets):
    correct_predictions = 0
    for y_pred, y_true in zip(predicted_outputs, targets):
        # Assuming a simple threshold to classify outputs
        predicted_label = 1 if y_pred > 0.5 else 0
        true_label = 1 if y_true > 0.5 else 0
        correct_predictions += (predicted_label == true_label)
    return correct_predictions / len(targets)

In [41]:
# Training

In [47]:
# Training
for epoch in range(num_epochs):
    initial_hidden_state = np.zeros((hidden_dim, 1))
    inputs = [np.random.randn(input_dim, 1) for _ in range(sequence_length)]
    targets = [np.random.randn(output_dim, 1) for _ in range(sequence_length)]
    
    predicted_outputs, targets = bptt(inputs, targets, initial_hidden_state)
    
    # Calculate loss (mean squared error)
    loss = np.mean((np.array(targets) - np.array(predicted_outputs)) ** 2)
    
    # Calculate accuracy
    accuracy = calculate_accuracy(predicted_outputs, targets)
    
    if epoch % 50 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

Epoch 0, Loss: 0.4727, Accuracy: 0.4000
Epoch 50, Loss: 0.4385, Accuracy: 0.4000
Epoch 100, Loss: 1.1671, Accuracy: 0.8000
Epoch 150, Loss: 1.4249, Accuracy: 0.6000
Epoch 200, Loss: 1.0969, Accuracy: 0.6000
Epoch 250, Loss: 0.7578, Accuracy: 0.6000
Epoch 300, Loss: 0.8443, Accuracy: 0.8000
Epoch 350, Loss: 0.4257, Accuracy: 0.8000
Epoch 400, Loss: 1.8166, Accuracy: 0.6000
Epoch 450, Loss: 0.5088, Accuracy: 0.6000
